# Homework1

In [1]:
import pandas as pd
import requests
import io
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import pickle
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import ttest_ind
from itertools import combinations
from collections import defaultdict
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy import sparse
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import kruskal


In [2]:
urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
]

dataframes = []
for url in urls:
    print(f"Reading from {url}")
    response = requests.get(url)
    response.raise_for_status()
    
    buffer = io.BytesIO(response.content)
    df = pd.read_parquet(buffer, engine="pyarrow") 
    df.columns = df.columns.str.lower()
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])


Reading from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Reading from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet


In [3]:
df.shape

(5980721, 19)

In [4]:
df['duration_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [10]:
std_duration=df[df['tpep_pickup_datetime'].dt.month == 1]['duration_minutes'].std()
print(f"Standard deviation of trip duration in January 2023: {std_duration:.2f}")

Standard deviation of trip duration in January 2023: 42.59


In [43]:
df_january = df[df['tpep_pickup_datetime'].dt.month == 1].copy()
df_january = df_january[(df_january['duration_minutes'] >= 1) & (df_january['duration_minutes'] <= 60)]

fraction_remaining = len(df_january) / len(df[df['tpep_pickup_datetime'].dt.month == 1])
print(f"Fraction remaining: {fraction_remaining:.2%}")


Fraction remaining: 98.12%


In [44]:
df_encoded = df_january[['pulocationid', 'dolocationid']].copy()
df_encoded = df_encoded.astype(str)
dicts = df_encoded.to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

print(f"Feature matrix shape: {X_train.shape}")

Feature matrix shape: (3009145, 515)


In [46]:
# Target variable
y_train = df_january['duration_minutes'].values

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and compute RMSE
y_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"Train RMSE: {rmse:.2f}")

# Save model and vectorizer
with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

Train RMSE: 7.65


In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)


df_feb = df[df['tpep_pickup_datetime'].dt.month == 2].copy()
df_feb = df_feb[(df_feb['duration_minutes'] >= 1) & (df_feb['duration_minutes'] <= 60)]


df_val_encoded = df_feb[['pulocationid', 'dolocationid']].copy()
df_val_encoded = df_val_encoded.astype(str)
dicts_val = df_val_encoded.to_dict(orient='records')
X_val = dv.transform(dicts_val)


y_val = df_feb['duration_minutes'].values
y_pred = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse_val:.2f}")


Validation RMSE: 7.81


# Test Model Building

In [7]:
df_train= df[df['tpep_pickup_datetime'].dt.month == 1].copy()
df_test= df[df['tpep_pickup_datetime'].dt.month == 2].copy()

# Feature Engineering

In [8]:
class TaxiFeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, df):
        df = df.copy()

        # Time-based
        df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
        df['pickup_dayofweek'] = df['tpep_pickup_datetime'].dt.dayofweek
        df['pickup_weekend'] = (df['pickup_dayofweek'] >= 5).astype(int)
        df['pickup_month'] = df['tpep_pickup_datetime'].dt.month

        # Route & interaction
        df['route'] = df['pulocationid'].astype(str) + '_' + df['dolocationid'].astype(str)
        df['pulocationid_x_hour'] = df['pulocationid'].astype(str) + '_' + df['pickup_hour'].astype(str)
        df['route_x_hour'] = df['route'] + '_' + df['pickup_hour'].astype(str)
        df['payment_type_x_ratecodeid'] = df['payment_type'].astype(str) + '_' + df['ratecodeid'].astype(str)

        # Speed & monetary ratios
        df['trip_duration_sec'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()
        df['trip_speed_mph'] = df['trip_distance'] / (df['trip_duration_sec'] / 3600 + 1e-5)
        df['tip_ratio'] = df['tip_amount'] / (df['fare_amount'] + 1e-5)
        df['fare_per_mile'] = df['fare_amount'] / (df['trip_distance'] + 1e-5)
        df['total_per_mile'] = df['total_amount'] / (df['trip_distance'] + 1e-5)

        return df


In [9]:
fe = TaxiFeatureEngineer()
df_train = fe.transform(df_train.copy())

## Features Selection

In [29]:
from pandas.api.types import is_numeric_dtype

def calculate_woe_iv(df, feature, target, bins=10, eps=1e-6):
    df = df[[feature, target]].dropna().copy()

    # Bin numeric variables only
    if is_numeric_dtype(df[feature]):
        try:
            df[feature] = pd.qcut(df[feature], q=bins, duplicates='drop')
        except Exception:
            return 0.0, pd.DataFrame()

    # Group and calculate event/non-event counts
    grouped = df.groupby(feature, observed=False)[target]
    woe_df = grouped.agg(['count', 'sum']).rename(columns={'sum': 'event'})
    woe_df['non_event'] = woe_df['count'] - woe_df['event']

    total_events = woe_df['event'].sum()
    total_non_events = woe_df['non_event'].sum()

    if total_events == 0 or total_non_events == 0:
        return 0.0, pd.DataFrame()

    # Calculate smoothed rates
    woe_df['event_rate'] = woe_df['event'] / total_events
    woe_df['non_event_rate'] = woe_df['non_event'] / total_non_events

    woe_df['event_rate'] = woe_df['event_rate'].replace(0, eps).fillna(eps)
    woe_df['non_event_rate'] = woe_df['non_event_rate'].replace(0, eps).fillna(eps)

    # Calculate WOE with warnings suppressed
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio = (woe_df['event_rate'] + eps) / (woe_df['non_event_rate'] + eps)
        ratio = ratio.replace([np.inf, -np.inf], eps).fillna(eps)
        woe_df['woe'] = np.log(ratio)

    woe_df['iv'] = (woe_df['event_rate'] - woe_df['non_event_rate']) * woe_df['woe']
    iv = woe_df['iv'].sum()

    return iv, woe_df[['woe', 'iv']]


In [30]:
categorical_features = df_train.select_dtypes(include=['object', 'category']).columns.tolist()


categorical_features = [col for col in categorical_features if col != 'duration_bin']
categorical_features += ['pulocationid', 'dolocationid']
df_train['duration_bin'] = pd.qcut(df_train['duration_minutes'], q=4, labels=False)


for col in df_train.select_dtypes(include=['int64', 'float64']).columns:
    if df_train[col].nunique() < 50 and col not in ['duration_minutes', 'duration_bin']:
        categorical_features.append(col)

df_train['duration_bin'] = pd.qcut(df_train['duration_minutes'], q=4, labels=False)

iv_results = []

print("Feature\t\tIV")
print("-" * 30)

for feature in categorical_features:
    try:
        iv, _ = calculate_woe_iv(df_train, feature, 'duration_bin')
        iv_results.append((feature, iv))
        print(f"{feature:<16}{iv:.4f}")
    except Exception as e:
        print(f"{feature:<16}Error: {e}")


Feature		IV
------------------------------
store_and_fwd_flag0.0000
route           0.3603
pulocationid_x_hour0.2425
route_x_hour    0.3739
payment_type_x_ratecodeid0.0720
pulocationid    0.1288
dolocationid    0.0768
vendorid        0.0000
passenger_count 0.0023
ratecodeid      0.0000
payment_type    0.0455
mta_tax         0.0000
improvement_surcharge0.0000
congestion_surcharge0.0000
airport_fee     0.1172
pickup_weekend  0.0000


In [8]:
# === Define feature groups ===
pearson_features = [
    'trip_distance',
    'fare_amount',
    'extra',
    'mta_tax',
    'tip_amount',
    'tolls_amount',
    'improvement_surcharge',
    'total_amount',
    'congestion_surcharge',
    'airport_fee',
    'trip_speed_mph',
    'tip_ratio',
    'fare_per_mile',
    'total_per_mile'
]

spearman_features = [
    'vendorid',
    'passenger_count',
    'ratecodeid',
    'store_and_fwd_flag',
    'pulocationid',
    'dolocationid',
    'payment_type',
    'pickup_hour',
    'pickup_dayofweek',
    'pickup_weekend',
    'pickup_month',
    'route',
    'pulocationid_x_hour',
    'route_x_hour',
    'payment_type_x_ratecodeid'
]

# Combine all for spearman
all_features = list(set(pearson_features + spearman_features))

# Make a safe copy
df_corr = df_train.copy()

# Label encode non-numeric spearman features
for col in spearman_features:
    if df_corr[col].dtype == 'object' or df_corr[col].dtype.name == 'category':
        df_corr[col] = LabelEncoder().fit_transform(df_corr[col].astype(str))

# === Calculate correlations ===

# Pearson correlation (numeric-only)
pearson_corr = df_corr[pearson_features + ['duration_minutes']].corr(method='pearson')['duration_minutes']

# Spearman correlation (ordinal + label-encoded categoricals)
spearman_corr = df_corr[pearson_features + spearman_features + ['duration_minutes']].corr(method='spearman')['duration_minutes']

# === Combine into one DataFrame ===

pearson_df = pd.DataFrame(pearson_corr).rename(columns={'duration_minutes': 'pearson'})
spearman_df = pd.DataFrame(spearman_corr).rename(columns={'duration_minutes': 'spearman'})

# Merge
correlation_df = pearson_df.join(spearman_df, how='outer')
correlation_df = correlation_df.drop(index='duration_minutes', errors='ignore')
correlation_df = correlation_df[~correlation_df.index.duplicated()]
correlation_df = correlation_df.sort_values(by='spearman', key=lambda x: x.abs(), ascending=False)

# Print result
print("📊 Correlation with duration_minutes:")
print(correlation_df.to_string(float_format="%.4f"))

📊 Correlation with duration_minutes:
                           pearson  spearman
fare_amount                 0.2091    0.9211
total_amount                0.2040    0.8891
trip_distance               0.0039    0.8504
total_per_mile             -0.0042   -0.6645
fare_per_mile              -0.0032   -0.5240
tip_amount                  0.1227    0.4218
tolls_amount                0.1264    0.3734
airport_fee                 0.1216    0.3554
ratecodeid                     NaN    0.2667
tip_ratio                  -0.0002   -0.2459
trip_speed_mph             -0.0009    0.1856
pulocationid_x_hour            NaN   -0.1301
route                          NaN   -0.1289
route_x_hour                   NaN   -0.1288
pulocationid                   NaN   -0.1193
dolocationid                   NaN   -0.1019
congestion_surcharge       -0.0378   -0.1004
payment_type                   NaN   -0.0671
extra                       0.0224    0.0654
improvement_surcharge       0.0091    0.0443
store_and_fwd_flag

In [7]:
# === Step 0: Define target ===
target = 'duration_minutes'

# === Step 1: Detect categorical columns ===
categorical_features = df_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Add manually known categorical ints
explicit_cat_ints = ['payment_type', 'ratecodeid', 'vendorid']
categorical_features += [col for col in explicit_cat_ints if col in df_train.columns]

# Add engineered categorical features
engineered_cats = [
    'route', 'pulocationid_x_hour', 'route_x_hour', 'payment_type_x_ratecodeid'
]
categorical_features += [col for col in engineered_cats if col in df_train.columns]

# Clean duplicates and exclude bins/targets
categorical_features = list(set(categorical_features))
categorical_features = [col for col in categorical_features if col not in ['duration_bin']]

# === Step 2: Split by cardinality ===
MAX_LEVELS = 100
low_card_cats = [col for col in categorical_features if df_train[col].nunique() <= MAX_LEVELS]
high_card_cats = [col for col in categorical_features if df_train[col].nunique() > MAX_LEVELS]

# === Step 3: Run ANOVA for low-cardinality ===
anova_results = {}
for col in low_card_cats:
    try:
        model = smf.ols(f'{target} ~ C({col})', data=df_train).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        pval = anova_table["PR(>F)"].iloc[0]
        anova_results[col] = pval
    except Exception as e:
        anova_results[col] = f"Error: {e}"

# === Step 4: Tree-based feature importance for high-cardinality ===

# Encode high-card categorical features
df_encoded = df_train[high_card_cats + [target]].dropna().copy()
for col in high_card_cats:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col].astype(str))

# Train random forest
X_tree = df_encoded[high_card_cats]
y_tree = df_encoded[target]
rf = RandomForestRegressor(n_estimators=50, max_depth=6, random_state=42, n_jobs=-1)
rf.fit(X_tree, y_tree)

# Collect importances
tree_importances = pd.Series(rf.feature_importances_, index=high_card_cats).sort_values(ascending=False)

# === Step 5: Display results ===

print("\n📊 ANOVA p-values (low-cardinality categorical features):")
sorted_anova = sorted(anova_results.items(), key=lambda x: x[1] if isinstance(x[1], float) else 1)
for feature, pval in sorted_anova:
    print(f"{feature:<30} {pval}")

print("\n🌲 Tree Feature Importances (high-cardinality categorical features):")
print(tree_importances.to_string(float_format="%.4f"))


📊 ANOVA p-values (low-cardinality categorical features):
ratecodeid                     0.0
payment_type_x_ratecodeid      0.0
payment_type                   1.1603530609144757e-187
vendorid                       1.4475528627943033e-164
store_and_fwd_flag             7.97553444401016e-11

🌲 Tree Feature Importances (high-cardinality categorical features):
route                 0.5779
route_x_hour          0.3359
pulocationid_x_hour   0.0862


In [9]:
# === Step 0: Define target ===
target = 'duration_minutes'

# === Step 1: Identify object/category columns ===
categorical_features = df_train.select_dtypes(include=['object', 'category']).columns.tolist()

# === Step 2: Add known numeric categoricals ===
explicit_cat_ints = ['payment_type', 'ratecodeid', 'vendorid']
categorical_features += [col for col in explicit_cat_ints if col in df_train.columns]

# === Step 3: Add engineered categoricals (new features) ===
engineered_cats = [
    'route', 'pulocationid_x_hour', 'route_x_hour', 'payment_type_x_ratecodeid'
]
categorical_features += [col for col in engineered_cats if col in df_train.columns]

# === Step 4: Deduplicate and filter for low-cardinality (≤ 6 groups) ===
categorical_features = list(set(categorical_features))
pairwise_features = [col for col in categorical_features if df_train[col].nunique() <= 6]

# === Step 5: Efficient pairwise T-tests ===
pairwise_results = defaultdict(list)

for feature in pairwise_features:
    col_data = df_train[[feature, target]].dropna()
    groups = col_data[feature].unique()

    for g1, g2 in combinations(groups, 2):
        group1 = col_data[col_data[feature] == g1][target]
        group2 = col_data[col_data[feature] == g2][target]

        # Welch's t-test
        t_stat, pval = ttest_ind(group1, group2, equal_var=False)
        pairwise_results[feature].append((g1, g2, pval))

# === Step 6: Display results ===
print("\n📊 Pairwise T-Test p-values for categorical features (low cardinality only):")
for feature, results in pairwise_results.items():
    print(f"\n{feature}:")
    for g1, g2, pval in results:
        sig = "✅" if pval < 0.05 else "❌"
        print(f"  {g1} vs {g2}: p = {pval:.4e} {sig}")


📊 Pairwise T-Test p-values for categorical features (low cardinality only):

vendorid:
  2 vs 1: p = 0.0000e+00 ✅

payment_type:
  2 vs 1: p = 1.6126e-10 ✅
  2 vs 4: p = 6.8061e-194 ✅
  2 vs 3: p = 1.7951e-265 ✅
  2 vs 0: p = 1.9533e-13 ✅
  1 vs 4: p = 2.8162e-195 ✅
  1 vs 3: p = 1.9669e-258 ✅
  1 vs 0: p = 1.0499e-126 ✅
  4 vs 3: p = 2.9472e-35 ✅
  4 vs 0: p = 2.3896e-305 ✅
  3 vs 0: p = 0.0000e+00 ✅

store_and_fwd_flag:
  N vs Y: p = 3.9435e-100 ✅


In [14]:
# === Step 0: Define target ===
target = 'duration_minutes'

# === Step 1: Define all candidate categorical features ===
categorical_features = [
    'payment_type', 'ratecodeid', 'vendorid', 'store_and_fwd_flag',
    'pulocationid', 'dolocationid',
    'route', 'pulocationid_x_hour', 'route_x_hour', 'payment_type_x_ratecodeid'
]

# === Step 2: Check cardinality and categorize ===
cardinality = {
    col: df_train[col].nunique()
    for col in categorical_features
}

cardinality_df = pd.DataFrame.from_dict(cardinality, orient='index', columns=['n_unique'])
cardinality_df['cardinality_level'] = pd.cut(
    cardinality_df['n_unique'],
    bins=[0, 10, 100, float('inf')],
    labels=['low', 'medium', 'high']
)
cardinality_df = cardinality_df.sort_values(by='n_unique')

# === Step 3: Filter safe features for MI ===
safe_mi_features = cardinality_df[cardinality_df['cardinality_level'].isin(['low', 'medium'])].index.tolist()
high_card_features = cardinality_df[cardinality_df['cardinality_level'] == 'high'].index.tolist()

# === Step 4: Prepare dataframe and encode only safe MI features ===
df_cat = df_train[safe_mi_features + [target]].dropna(subset=[target]).copy()

# Optional: Sample if very large
MAX_ROWS = 200_000
if len(df_cat) > MAX_ROWS:
    df_cat = df_cat.sample(n=MAX_ROWS, random_state=42)

# Label encode safe features
for col in safe_mi_features:
    df_cat[col] = LabelEncoder().fit_transform(df_cat[col].astype(str))

X_cat = df_cat[safe_mi_features]
y_cat = df_cat[target]

# === Step 5: Compute MI ===
mi_scores = mutual_info_regression(X_cat, y_cat, discrete_features=True)
mi_df = pd.DataFrame({'Feature': safe_mi_features, 'MI_Score': mi_scores})
mi_df = mi_df.sort_values(by='MI_Score', ascending=False).reset_index(drop=True)

# === Step 6: Output results ===
print("📊 Mutual Information Scores (Low & Medium Cardinality Features):")
print(mi_df.to_string(index=False, float_format="%.4f"))

print("\n⚠️ Skipped High Cardinality Features (Consider alternatives):")
for col in high_card_features:
    n = cardinality[col]
    print(f"  {col:<25} ({n} unique) → use RandomForest, Target Encoding, or Binning")


📊 Mutual Information Scores (Low & Medium Cardinality Features):
                  Feature  MI_Score
payment_type_x_ratecodeid    0.1311
               ratecodeid    0.1173
             payment_type    0.0204
       store_and_fwd_flag    0.0097
                 vendorid    0.0023

⚠️ Skipped High Cardinality Features (Consider alternatives):
  pulocationid              (257 unique) → use RandomForest, Target Encoding, or Binning
  dolocationid              (261 unique) → use RandomForest, Target Encoding, or Binning
  pulocationid_x_hour       (5111 unique) → use RandomForest, Target Encoding, or Binning
  route                     (22754 unique) → use RandomForest, Target Encoding, or Binning
  route_x_hour              (159394 unique) → use RandomForest, Target Encoding, or Binning


In [16]:
# === Target ===
target = 'duration_minutes'

# === Define features ===

# Features that passed MI test (medium cardinality)
mi_passed_features = [
    'payment_type_x_ratecodeid', 'ratecodeid', 'payment_type',
    'store_and_fwd_flag', 'vendorid'
]

# High-cardinality features
high_card_features = [
    'pulocationid', 'dolocationid',
    'pulocationid_x_hour', 'route', 'route_x_hour'
]

# Combine and ensure unique
all_features = list(set(mi_passed_features + high_card_features))

# === Optional: Sample for performance
df_sample = df_train[all_features + [target]].dropna(subset=[target]).copy()
if len(df_sample) > 200_000:
    df_sample = df_sample.sample(200_000, random_state=42)

# === Function to run Kruskal–Wallis Test ===
def run_kruskal_test(df, feature, target, max_groups=50, min_group_size=20):
    vc = df[feature].value_counts()
    top_vals = vc[vc >= min_group_size].nlargest(max_groups).index
    groups = [df[df[feature] == val][target] for val in top_vals]

    # Require at least 2 groups
    if len(groups) < 2:
        return None, None

    stat, pval = kruskal(*groups)
    return stat, pval

# === Run test for each feature ===
results = []
for feat in all_features:
    stat, pval = run_kruskal_test(df_sample, feat, target)
    if stat is not None:
        results.append((feat, stat, pval))

# === Format and display ===
results_df = pd.DataFrame(results, columns=['Feature', 'H_statistic', 'p_value'])
results_df['Significant'] = results_df['p_value'] < 0.05
results_df = results_df.sort_values(by='p_value')

print("📊 Kruskal–Wallis H-test Results (Top 50 values per feature):")
print(results_df.to_string(index=False, float_format="%.4e"))


📊 Kruskal–Wallis H-test Results (Top 50 values per feature):
                  Feature  H_statistic     p_value  Significant
               ratecodeid   1.8095e+04  0.0000e+00         True
      pulocationid_x_hour   1.0259e+04  0.0000e+00         True
payment_type_x_ratecodeid   2.0489e+04  0.0000e+00         True
             dolocationid   1.0219e+04  0.0000e+00         True
                    route   9.0662e+03  0.0000e+00         True
             payment_type   1.6409e+03  0.0000e+00         True
             pulocationid   2.8386e+04  0.0000e+00         True
             route_x_hour   1.2573e+03 4.6183e-231         True
                 vendorid   1.8166e+01  2.0249e-05         True
       store_and_fwd_flag   1.1699e+00  2.7941e-01        False


In [18]:
numeric_features = [
    'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
    'tolls_amount', 'improvement_surcharge', 'total_amount',
    'congestion_surcharge', 'airport_fee',
    'trip_duration_sec', 'trip_speed_mph',
    'tip_ratio', 'fare_per_mile', 'total_per_mile'
]


# Prepare DataFrame: drop NaNs
X_vif = df_train[numeric_features].dropna().copy()

# Add constant (required by statsmodels)
X_vif = add_constant(X_vif)

# Compute VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# Display results
print("\n📊 Variance Inflation Factor (VIF) for Numeric Features:")
print(vif_data.to_string(index=False, float_format='%.2f'))


📊 Variance Inflation Factor (VIF) for Numeric Features:
              Feature    VIF
                const  29.73
        trip_distance   1.01
          fare_amount 463.97
                extra   2.70
              mta_tax   4.27
           tip_amount  24.17
         tolls_amount   8.55
improvement_surcharge   4.28
         total_amount 723.61
 congestion_surcharge   2.59
          airport_fee   2.63
    trip_duration_sec   1.05
       trip_speed_mph   1.00
            tip_ratio   1.01
        fare_per_mile  41.91
       total_per_mile  41.22


# ✅ Final Feature Selection Summary

## 📂 Categorical Features (Selected)

| Feature                  | IV     | MI Score | ANOVA P-value        | Kruskal H-test | T-Test    | Reason for Selection |
|--------------------------|--------|----------|----------------------|----------------|-----------|-----------------------|
| `pulocationid`           | 0.1288 | —        | —                    | ✅ (28,386)     | —         | High IV, Kruskal significant |
| `dolocationid`           | 0.0768 | —        | —                    | ✅ (10,219)     | —         | High IV, Kruskal significant |
| `ratecodeid`             | 0.0000 | 0.1173   | ✅ (0.0)              | ✅              | —         | Strong MI + Kruskal + ANOVA |
| `payment_type`           | 0.0455 | 0.0204   | ✅ (very low)         | ✅              | ✅         | Strong across all tests |
| `store_and_fwd_flag`     | 0.0000 | 0.0097   | ✅ (7.98e-11)         | ❌              | ✅         | Low IV/MI but t-test significant |
| `payment_type_x_ratecodeid` | 0.0720 | 0.1311 | ✅ (0.0)           | ✅              | —         | High MI + ANOVA + Kruskal |
| `pulocationid_x_hour`    | 0.2425 | —        | —                    | ✅ (10,259)     | —         | High IV + tree importance + Kruskal |
| `route`                  | 0.3603 | —        | —                    | ✅ (9,066)      | —         | High IV + tree importance |
| `route_x_hour`           | 0.3739 | —        | —                    | ✅ (1,257)      | —         | Very high IV + tree importance |

## 🚫 Dropped Categorical Features

| Feature              | IV     | MI Score | Reason for Dropping                         |
|----------------------|--------|----------|---------------------------------------------|
| `vendorid`           | 0.0000 | 0.0023   | Weak in IV + very low MI + low effect size |
| `passenger_count`    | 0.0023 | —        | Weak correlation and not stat sig overall   |

---

## 📊 Numerical Features (Selected)

| Feature                | VIF   | Spearman Corr | Pearson Corr | Reason for Selection |
|------------------------|-------|----------------|----------------|-----------------------|
| `trip_distance`        | 1.01  | 0.8504         | 0.0039         | Very strong monotonic trend, low VIF |
| `extra`                | 2.70  | 0.0654         | 0.0224         | Independent cost, low VIF |
| `congestion_surcharge` | 2.59  | -0.1004        | -0.0378        | Weak inverse trend, low collinearity |
| `airport_fee`          | 2.63  | 0.3554         | 0.1216         | Useful signal, no multicollinearity |

## 🚫 Dropped Numerical Features

| Feature                  | VIF     | Reason for Dropping                                  |
|--------------------------|---------|------------------------------------------------------|
| `fare_amount`            | 463.97  | Redundant with total_amount, multicollinearity      |
| `total_amount`           | 723.61  | Aggregated collinear metric, inflated VIF           |
| `tip_amount`             | 24.17   | Overlaps with fare_amount, high VIF                 |
| `tolls_amount`           | 8.55    | Optional for non-linear models, moderate VIF        |
| `mta_tax`                | 4.27    | Very weak signal and small variance                 |
| `improvement_surcharge` | 4.28    | Weak signal + moderate VIF                          |
| `fare_per_mile`         | 41.91   | Correlated with fare and trip_distance              |
| `total_per_mile`        | 41.22   | High VIF + redundancy with fare/total amount        |

---

## 🧾 Final Feature Lists

```python
cat_features = [
    'pulocationid', 'dolocationid', 'ratecodeid', 'payment_type',
    'store_and_fwd_flag', 'payment_type_x_ratecodeid',
    'pulocationid_x_hour', 'route', 'route_x_hour'
]

num_features = [
    'trip_distance', 'extra', 'congestion_surcharge', 'airport_fee'
]


In [10]:
cat_features = [
    'pulocationid', 'dolocationid', 'ratecodeid', 'payment_type',
    'store_and_fwd_flag', 'payment_type_x_ratecodeid',
    'pulocationid_x_hour', 'route', 'route_x_hour'
]

num_features = [
    'trip_distance', 'extra', 'congestion_surcharge', 'airport_fee'
]
df_train= df_train[(df_train['duration_minutes'] >= 1) & (df_train['duration_minutes'] <= 60)]

In [11]:
selected_cols = cat_features + num_features + ['duration_minutes']

# Prepare and clean data
df_train_subset = df_train[selected_cols].dropna().copy()
df_train_subset[cat_features] = df_train_subset[cat_features].astype(str)

# Vectorize only categorical features as sparse
dv = DictVectorizer(sparse=True)
cat_dicts = df_train_subset[cat_features].to_dict(orient='records')
X_cat = dv.fit_transform(cat_dicts)  # sparse matrix

# Get numeric features
X_num = df_train_subset[num_features].values

# Combine sparse categorical and dense numeric
X_train = sparse.hstack([X_cat, X_num]).tocsr()

# Extract target
y_train = df_train_subset['duration_minutes'].values

In [12]:
# 1. Initialize the model
model = LinearRegression()

# 2. Train on training data
model.fit(X_train, y_train)

# 3. Predict on the same training set
y_pred = model.predict(X_train)

# 4. Evaluate with RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"✅ Train RMSE: {rmse:.2f}")

✅ Train RMSE: 4.32


In [ ]:
df_test = df_test[(df_test['duration_minutes'] >= 1) & (df_test['duration_minutes'] <= 60)]


df_test = fe.transform(df_test.copy())

df_test_subset = df_test[selected_cols].dropna().copy()

df_test_subset[cat_features] = df_test_subset[cat_features].astype(str)

cat_dicts_test = df_test_subset[cat_features].to_dict(orient='records')
X_cat_test = dv.transform(cat_dicts_test) 

X_num_test = df_test_subset[num_features].values

X_test = sparse.hstack([X_cat_test, X_num_test]).tocsr()

y_test = df_test_subset['duration_minutes'].values

y_test_pred = model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"🧪 Test RMSE: {rmse_test:.2f}")


🧪 Test RMSE: 4.75
